Subtask B: Verifiable factual claims detection: Given a tweet, predict whether it contains a verifiable factual claim. This is a binary task with two labels: Yes and No. This is a classification task

In [152]:
import pandas as pd
import numpy as np
import json
import nltk
from nltk import ngrams
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import spacy

import warnings
warnings.filterwarnings("ignore")

# Load Lemmatizer
nlp = spacy.load("en_core_web_sm")

# Load Stopwords
nltk.download('stopwords')
cachedStopWords = list(set(stopwords.words("english")))



# a)
# Twitter Datasets
df_worthy_train = pd.read_csv("data/1b/CT22_english_1B_claim_train.tsv", sep='\t')
df_worthy_test = pd.read_csv("data/1b/CT22_english_1B_claim_dev_test.tsv", sep='\t')

# Top 3000 most commonly used english words + covid related medical terms (used as vocabulary for the vectorization process)
topwords = pd.read_csv('words.txt', sep=" ", header=None).values[:,0]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\minh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Handling of imbalanced data

In [153]:
df_worthy_train["class_label"].value_counts()

1    2122
0    1202
Name: class_label, dtype: int64

In [154]:
df_true = df_worthy_train[df_worthy_train["class_label"] == 1]
df_true.shape

(2122, 5)

In [155]:
df_false = df_worthy_train[df_worthy_train["class_label"] == 0]
df_false.shape

(1202, 5)

In [156]:
df_true_downsampled = df_true.sample(df_false.shape[0])
df_true_downsampled.shape

(1202, 5)

In [157]:
# downsampled
df_balanced_d = pd.concat([df_true_downsampled, df_false])
df_balanced_d.shape

(2404, 5)

In [158]:
df_false_upsampled = df_false.sample(df_true.shape[0], replace=True)
df_false_upsampled.shape

(2122, 5)

In [159]:
# upsampled
df_balanced_u = pd.concat([df_true, df_false_upsampled])
df_balanced_u.shape

(4244, 5)

In [160]:
mean_val = (df_false.shape[0] + df_true.shape[0])/2
df_true_mv = df_true.sample(int(mean_val))
df_false_mv = df_false.sample(int(mean_val), replace=True)

In [161]:
# Hybrid
df_balanced_h = pd.concat([df_true_mv, df_false_mv])
df_balanced_h.shape

(3324, 5)

### Emoji Removal

In [162]:
# Function to remove emojies when using the lemmatizer
#https://poopcode.com/how-to-remove-emoji-from-text-in-python/
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

### Custom Tokenizer

In [163]:
def tokenize_text(text, method="stopwords"):
    
    # Remove hyperlinks
    text = re.sub(r'http\S+', '', text)
    
    # Split sentence into words
    tokens = []
    for token in ngrams(text.split(), 1):
        word = re.sub(r',', '', token[0]) #Remove commas
        word = re.sub(r'[\!\.\:]$', '', word) #Remove (.!:) at the end of tokens
        word = re.sub(r'#', '', word) #Remove hashtags (not the text)
        if word == "—": continue #Ignore dash
        if word.find("@") != -1: continue #Ignore tags
        tokens.append(word)
    tokens

    # List comprehension to remove stopwords
    if method != "default":
        tokens = [x for x in tokens if x not in cachedStopWords]

    # Perform stemming
    if method == "stemming":
        ps = PorterStemmer()
        tokens = [ps.stem(x) for x in tokens]

    # Perform lemmatization
    if method == "lemmatizer":
        # Remove emojies due to lemmatizer not handling them well
        tokens = [x for x in tokens if remove_emoji(x) == x]
        text = " ".join(tokens)
        doc = nlp(text)
        tokens = [token.lemma_ for token in doc]

    return tokens

In [164]:
def tokenize_normal(text):
    return tokenize_text(text, method="default")

def tokenize_stopwords(text):
    return tokenize_text(text, method="stopwords")

def tokenize_stemming(text):
    return tokenize_text(text, method="stemming")

def tokenize_lemmatizer(text):
    return tokenize_text(text, method="lemmatizer")

In [165]:
def tokenize_df(df, text_col, method="default"):
    article_tokens = []
    for i in range(len(df)):
        text = df.iloc[i][text_col].lower()
        article_tokens.append(tokenize_text(text, method))
    df["tokens"] = article_tokens

In [166]:
tokenize_df(df_balanced_d, "tweet_text", "lemmatizer")
tokenize_df(df_balanced_u, "tweet_text", "lemmatizer")
tokenize_df(df_balanced_h, "tweet_text", "lemmatizer")
tokenize_df(df_worthy_train, "tweet_text", "lemmatizer")
tokenize_df(df_worthy_test, "tweet_text", "lemmatizer")

In [167]:
print(df_balanced_d.iloc[0]["tokens"], "\n")
print(df_balanced_u.iloc[0]["tokens"], "\n")
print(df_balanced_h.iloc[0]["tokens"], "\n")
print(df_worthy_train.iloc[0]["tokens"], "\n")
print(df_worthy_test.iloc[0]["tokens"])

['tony', 'fauci', 'swine', 'flu', 'hoax', ';', 'betrayal', 'trust', 'covid-19', 'first', 'time', 'fauci', 'cdc', 'lie', 'epidemic', 'push', 'dangerously', 'rush', '"', 'vaccine', '"', 'covid19', 'covidvaccine'] 

['india', "'s", 'gift', '100000', 'covid-19', 'vaccine', 'arrive', 'barbado', 'early', 'today', 'special', 'moment', 'barbadian', 'want', 'thank', 'prime', 'minister', 'modi', 'quick', 'decisive', 'magnanimous', 'action', 'allow', 'we', 'beneficiary', 'vaccine'] 

['china', 'country', 'say', 'launch', 'new', 'programme', 'provide', 'million', 'covid-19', 'vaccine', 'national', 'africa', 'elsewhere', 'overseas', 'accord', 'foreign', 'minister', 'wang', 'yi'] 

['india', "'s", 'gift', '100000', 'covid-19', 'vaccine', 'arrive', 'barbado', 'early', 'today', 'special', 'moment', 'barbadian', 'want', 'thank', 'prime', 'minister', 'modi', 'quick', 'decisive', 'magnanimous', 'action', 'allow', 'we', 'beneficiary', 'vaccine'] 

['senate', 'pass', 'covid', 'relief', '$', '1400', 'relief

### Bag of words

In [168]:
from sklearn.feature_extraction.text import CountVectorizer # For creating a DTM (discrete values)
from sklearn.feature_extraction.text import TfidfVectorizer # A weighted DTM (fractions)

cv_dict = {}
tfidf_dict = {}
ngram_list = [1,2,3]
for n in ngram_list:
    cv_dict[n] = CountVectorizer(tokenizer=tokenize_lemmatizer, ngram_range=(n, n), max_features=5000)
    tfidf_dict[n] = TfidfVectorizer(tokenizer=tokenize_stopwords, ngram_range=(n, n), max_features=5000)
#cv = CountVectorizer(tokenizer=tokenize_lemmatizer, ngram_range=(1, 1), max_features=5000) #Test with different n-grams, min_df, max_df and max_features
#tfidf = TfidfVectorizer(tokenizer=tokenize_stopwords, ngram_range=(1, 1), max_features=5000) #Test with (1,1), (2,2), and (3,3) 

In [169]:
tweet_vec = {}
tweet_bow_train = {}
tweet_bow_test = {}

tweet_vec_d = {}
tweet_bow_train_d = {}
tweet_bow_test_d = {}

tweet_vec_u = {}
tweet_bow_train_u = {}
tweet_bow_test_u = {}

tweet_vec_h = {}
tweet_bow_train_h = {}
tweet_bow_test_h = {}
for n in ngram_list:
    tweet_vec[n] = cv_dict[n].fit(df_worthy_train["tweet_text"]) # DTM (CV, but normalized for relative frequency)
    tweet_bow_train[n] = tweet_vec[n].transform(df_worthy_train["tweet_text"]) 
    tweet_bow_test[n] = tweet_vec[n].transform(df_worthy_test["tweet_text"]) 
    
    # balanced downsample
    tweet_vec_d[n] = cv_dict[n].fit(df_balanced_d["tweet_text"]) # DTM (CV, but normalized for relative frequency)
    tweet_bow_train_d[n] = tweet_vec_d[n].transform(df_balanced_d["tweet_text"]) 
    tweet_bow_test_d[n] = tweet_vec_d[n].transform(df_worthy_test["tweet_text"]) 
    
    # balanced upsample
    tweet_vec_u[n] = cv_dict[n].fit(df_balanced_u["tweet_text"]) # DTM (CV, but normalized for relative frequency)
    tweet_bow_train_u[n] = tweet_vec_u[n].transform(df_balanced_u["tweet_text"]) 
    tweet_bow_test_u[n] = tweet_vec_u[n].transform(df_worthy_test["tweet_text"]) 
    
    # balanced hybridsample
    tweet_vec_h[n] = cv_dict[n].fit(df_balanced_h["tweet_text"]) # DTM (CV, but normalized for relative frequency)
    tweet_bow_train_h[n] = tweet_vec_h[n].transform(df_balanced_h["tweet_text"]) 
    tweet_bow_test_h[n] = tweet_vec_h[n].transform(df_worthy_test["tweet_text"]) 

### Feature Exploration

In [170]:
# If max_features not limited, there are 8263 unique words (features) and a total of 51333 words across 2123 tweets
for n in ngram_list:
    captured = tweet_bow_train[n].sum()
    print(f"Total words across all tweets: 51333")
    print(f"Total words captured by our features/vocabulary {captured} ({round(100*captured/51333, 1)}%)")
    print(f"Amount of unique words (tokens/features): {len(tweet_vec[n].vocabulary_)}\n") # Unique words (features)

    # vec.vocabulary_items() => Dict with every word in the vocabulary with the DTM column index as the value
    sum_words = tweet_bow_train[n].sum(axis=0) # Sum the columns from the DTM matrix
    words_freq = [(word, sum_words[0, index]) for word, index in tweet_vec[n].vocabulary_.items()] # Make a list pairing every word with its corresponding DTM column summation
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
    print("Most frequent terms for {}-gram: \n {}".format(n, words_freq[:10])) # show top n terms used

Total words across all tweets: 51333
Total words captured by our features/vocabulary 67277 (131.1%)
Amount of unique words (tokens/features): 5000

Most frequent terms for 1-gram: 
 [('wasteful', 3853), ('door', 2047), ('-', 962), ('"', 590), ('icu', 558), ('donation', 491), ('precaution', 404), ('dose', 371), ('ab', 351), ('worry', 348)]
Total words across all tweets: 51333
Total words captured by our features/vocabulary 22939 (44.7%)
Amount of unique words (tokens/features): 5000

Most frequent terms for 2-gram: 
 [('covid-19 talk', 1250), ('$ 500', 294), ('; that', 294), ('country access', 247), ('covid19 reminder', 140), ('covid-19 test', 104), ('- attack', 100), ('roll vaccine', 89), ('community -', 79), ('he say', 72)]
Total words across all tweets: 51333
Total words captured by our features/vocabulary 9921 (19.3%)
Amount of unique words (tokens/features): 5000

Most frequent terms for 3-gram: 
 [("' cure '", 294), ('covid-19 vaccine support', 56), ('administer vaccine quickly', 

### Dummy Predictions

In [171]:
from sklearn.dummy import DummyClassifier
from sklearn import metrics

for n in ngram_list:
    print("--------Unbalanced--------")
    dummy_clf = DummyClassifier(strategy="prior")
    dummy_clf.fit(tweet_bow_train[n], df_worthy_train["class_label"])
    print("{}-gram Priors: {}".format(n, dummy_clf.class_prior_))

    accuracy_score = metrics.accuracy_score(dummy_clf.predict(tweet_bow_test[n]), df_worthy_test["class_label"])
    print("Dummy Accuracy for {}-gram: {}%".format(n, round(accuracy_score * 100, 2)))

    f1_score = metrics.f1_score(dummy_clf.predict(tweet_bow_test[n]), df_worthy_test["class_label"])
    print("Dummy F1-Score for {}-gram: {}%".format(n, round(f1_score * 100, 2)))
    
    # Downsampled
    print("--------Downsampled--------")
    dummy_clf_d = DummyClassifier(strategy="prior")
    dummy_clf_d.fit(tweet_bow_train_d[n], df_balanced_d["class_label"])
    print("{}-gram Priors: {}".format(n, dummy_clf_d.class_prior_))

    accuracy_score = metrics.accuracy_score(dummy_clf_d.predict(tweet_bow_test_d[n]), df_worthy_test["class_label"])
    print("Dummy Accuracy for {}-gram: {}%".format(n, round(accuracy_score * 100, 2)))

    f1_score = metrics.f1_score(dummy_clf_d.predict(tweet_bow_test_d[n]), df_worthy_test["class_label"])
    print("Dummy F1-Score for {}-gram: {}%".format(n, round(f1_score * 100, 2)))
    
    # Upsampled
    print("--------Upsampled--------")
    dummy_clf_u = DummyClassifier(strategy="prior")
    dummy_clf_u.fit(tweet_bow_train_u[n], df_balanced_u["class_label"])
    print("{}-gram Priors: {}".format(n, dummy_clf_u.class_prior_))

    accuracy_score = metrics.accuracy_score(dummy_clf_u.predict(tweet_bow_test_u[n]), df_worthy_test["class_label"])
    print("Dummy Accuracy for {}-gram: {}%".format(n, round(accuracy_score * 100, 2)))

    f1_score = metrics.f1_score(dummy_clf_u.predict(tweet_bow_test_u[n]), df_worthy_test["class_label"])
    print("Dummy F1-Score for {}-gram: {}%".format(n, round(f1_score * 100, 2)))
    
    # Hybridsampled
    print("--------Hybridsampled--------")
    dummy_clf_h = DummyClassifier(strategy="prior")
    dummy_clf_h.fit(tweet_bow_train_h[n], df_balanced_h["class_label"])
    print("{}-gram Priors: {}".format(n, dummy_clf_h.class_prior_))

    accuracy_score = metrics.accuracy_score(dummy_clf_h.predict(tweet_bow_test_h[n]), df_worthy_test["class_label"])
    print("Dummy Accuracy for {}-gram: {}%".format(n, round(accuracy_score * 100, 2)))

    f1_score = metrics.f1_score(dummy_clf_h.predict(tweet_bow_test_h[n]), df_worthy_test["class_label"])
    print("Dummy F1-Score for {}-gram: {}%".format(n, round(f1_score * 100, 2)))
    print("--------------------------------------")

--------Unbalanced--------
1-gram Priors: [0.36161252 0.63838748]
Dummy Accuracy for 1-gram: 63.01%
Dummy F1-Score for 1-gram: 77.31%
--------Downsampled--------
1-gram Priors: [0.5 0.5]
Dummy Accuracy for 1-gram: 36.99%
Dummy F1-Score for 1-gram: 0.0%
--------Upsampled--------
1-gram Priors: [0.5 0.5]
Dummy Accuracy for 1-gram: 36.99%
Dummy F1-Score for 1-gram: 0.0%
--------Hybridsampled--------
1-gram Priors: [0.5 0.5]
Dummy Accuracy for 1-gram: 36.99%
Dummy F1-Score for 1-gram: 0.0%
--------------------------------------
--------Unbalanced--------
2-gram Priors: [0.36161252 0.63838748]
Dummy Accuracy for 2-gram: 63.01%
Dummy F1-Score for 2-gram: 77.31%
--------Downsampled--------
2-gram Priors: [0.5 0.5]
Dummy Accuracy for 2-gram: 36.99%
Dummy F1-Score for 2-gram: 0.0%
--------Upsampled--------
2-gram Priors: [0.5 0.5]
Dummy Accuracy for 2-gram: 36.99%
Dummy F1-Score for 2-gram: 0.0%
--------Hybridsampled--------
2-gram Priors: [0.5 0.5]
Dummy Accuracy for 2-gram: 36.99%
Dummy F1-Sc

### Naive Bayes Prediction

In [172]:
#https://scikit-learn.org/stable/modules/naive_bayes.html#naive-bayes
from sklearn.naive_bayes import MultinomialNB # suited for classification with discrete features
from sklearn.naive_bayes import ComplementNB # suited for imbalanced datasets (designed to correct for "severe assumptions")
from sklearn.naive_bayes import BernoulliNB # designed for classification with binary/boolean features
from sklearn.naive_bayes import GaussianNB # designed for classification with continuous values

clf_nb = ComplementNB()  # Unbalanced
clf_d = MultinomialNB()   # Balanced
clf_u = MultinomialNB() 
clf_h = MultinomialNB() 
for n in ngram_list:
    print("--------Unbalanced--------")
    clf_nb.fit(tweet_bow_train[n].toarray(), df_worthy_train["class_label"])

    accuracy_score = metrics.accuracy_score(clf_nb.predict(tweet_bow_test[n].toarray()), df_worthy_test["class_label"])
    print("Complement Accuracy for {}-gram: {}%".format(n, round(accuracy_score * 100, 2)))

    accuracy_score = metrics.f1_score(clf_nb.predict(tweet_bow_test[n].toarray()), df_worthy_test["class_label"])
    print("Complement F1-Score for {}-gram: {}%".format(n, round(accuracy_score * 100, 2)))
    
    # Balanced
    print("--------Downsampled--------")
    clf_d.fit(tweet_bow_train_d[n].toarray(), df_balanced_d["class_label"])
    
    accuracy_score = metrics.accuracy_score(clf_d.predict(tweet_bow_test_d[n].toarray()), df_worthy_test["class_label"])
    print("Multinomial Accuracy for {}-gram: {}%".format(n, round(accuracy_score * 100, 2)))

    accuracy_score = metrics.f1_score(clf_d.predict(tweet_bow_test_d[n].toarray()), df_worthy_test["class_label"])
    print("Multinomial F1-Score for {}-gram: {}%".format(n, round(accuracy_score * 100, 2)))
    
    # Balanced
    print("--------Upsampled--------")
    clf_u.fit(tweet_bow_train_u[n].toarray(), df_balanced_u["class_label"])
    
    accuracy_score = metrics.accuracy_score(clf_u.predict(tweet_bow_test_u[n].toarray()), df_worthy_test["class_label"])
    print("Multinomial Accuracy for {}-gram: {}%".format(n, round(accuracy_score * 100, 2)))

    accuracy_score = metrics.f1_score(clf_u.predict(tweet_bow_test_u[n].toarray()), df_worthy_test["class_label"])
    print("Multinomial F1-Score for {}-gram: {}%".format(n, round(accuracy_score * 100, 2)))
    
    # Balanced
    print("--------Hybridsampled--------")
    clf_h.fit(tweet_bow_train_h[n].toarray(), df_balanced_h["class_label"])
    
    accuracy_score = metrics.accuracy_score(clf_h.predict(tweet_bow_test_h[n].toarray()), df_worthy_test["class_label"])
    print("Multinomial Accuracy for {}-gram: {}%".format(n, round(accuracy_score * 100, 2)))

    accuracy_score = metrics.f1_score(clf_h.predict(tweet_bow_test_h[n].toarray()), df_worthy_test["class_label"])
    print("Multinomial F1-Score for {}-gram: {}%".format(n, round(accuracy_score * 100, 2)))
    print("--------------------------------------")

--------Unbalanced--------
Complement Accuracy for 1-gram: 72.56%
Complement F1-Score for 1-gram: 77.52%
--------Downsampled--------
Multinomial Accuracy for 1-gram: 70.8%
Multinomial F1-Score for 1-gram: 74.71%
--------Upsampled--------
Multinomial Accuracy for 1-gram: 71.68%
Multinomial F1-Score for 1-gram: 77.68%
--------Hybridsampled--------
Multinomial Accuracy for 1-gram: 71.68%
Multinomial F1-Score for 1-gram: 77.68%
--------------------------------------
--------Unbalanced--------
Complement Accuracy for 2-gram: 64.65%
Complement F1-Score for 2-gram: 69.85%
--------Downsampled--------
Multinomial Accuracy for 2-gram: 61.69%
Multinomial F1-Score for 2-gram: 65.62%
--------Upsampled--------
Multinomial Accuracy for 2-gram: 65.09%
Multinomial F1-Score for 2-gram: 76.13%
--------Hybridsampled--------
Multinomial Accuracy for 2-gram: 63.67%
Multinomial F1-Score for 2-gram: 75.32%
--------------------------------------
--------Unbalanced--------
Complement Accuracy for 3-gram: 52.8%


### Word Embedding

In [173]:
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
  
warnings.filterwarnings(action = 'ignore')
  
import gensim
from gensim.models import Word2Vec

In [174]:
model = Word2Vec(df_balanced_d["tokens"].values, min_count=1)

In [175]:
from sklearn.decomposition import PCA

X = model[model.wv.vocab]
pca = PCA(n_components=3)

result = pca.fit_transform(X)

In [176]:
from matplotlib import pyplot
# https://github.com/LogicalShark/wordvec/blob/master/wvplot.py

axes = [0, 1, 2]

colors = ["tab:red", "tab:blue", "tab:green", "tab:orange",
          "tab:purple", "tab:olive", "tab:pink", "tab:cyan", "tab:gray"]
defaultcolor = "black"

sizes = []
defaultsize = 16

def plot3D(result, wordgroups):
    fig = pyplot.figure(figsize=(12, 12))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(result[:, axes[0]], result[:, axes[1]], result[:, axes[2]])
    for g, group in enumerate(wordgroups):
        for word in group:
            if not word in words:
                continue
            i = words.index(word)
            # Create plot point
            color = colors[g] if g < len(colors) else defaultcolor
            size = sizes[g] if g < len(sizes) else defaultsize
            ax.text(result[i, axes[0]], result[i, axes[1]],
                    result[i, axes[2]], word, color=color, fontsize=size)

In [177]:
import matplotlib.pyplot as plt
words = list(model.wv.vocab)

plot3D(result, words)